In [1]:
import pandas as pd

In [2]:
pokemon_cards = pd.read_csv("pokemon_cards.csv", encoding="latin1")
pokemon_tcg_data = pd.read_csv("pokemon-tcg-data-master 1999-2023.csv", encoding="latin1")

In [3]:
print(pokemon_cards.head())

    Pokemon     Card Type Generation Card Number  Price £
0  AMPHAROS  REVERSE HOLO  AQUAPOLIS  001 OF 147     2.95
1  AMPHAROS      STANDARD  AQUAPOLIS  001 OF 147     2.50
2  ARCANINE  REVERSE HOLO  AQUAPOLIS  002 OF 147     3.95
3  ARCANINE      STANDARD  AQUAPOLIS  002 OF 147     2.95
4   ARIADOS  REVERSE HOLO  AQUAPOLIS  003 OF 147     2.50


In [4]:
print(pokemon_tcg_data.head())

        id   set series publisher generation release_date           artist  \
0  base1-1  Base   Base      WOTC      First     1/9/1999     Ken Sugimori   
1  base1-2  Base   Base      WOTC      First     1/9/1999     Ken Sugimori   
2  base1-3  Base   Base      WOTC      First     1/9/1999     Ken Sugimori   
3  base1-4  Base   Base      WOTC      First     1/9/1999  Mitsuhiro Arita   
4  base1-5  Base   Base      WOTC      First     1/9/1999     Ken Sugimori   

        name set_num          types  ...  \
0   Alakazam       1    ['Psychic']  ...   
1  Blastoise       2      ['Water']  ...   
2    Chansey       3  ['Colorless']  ...   
3  Charizard       4       ['Fire']  ...   
4   Clefairy       5  ['Colorless']  ...   

                               retreatCost convertedRetreatCost     rarity  \
0  ['Colorless', 'Colorless', 'Colorless']                  3.0  Rare Holo   
1  ['Colorless', 'Colorless', 'Colorless']                  3.0  Rare Holo   
2                            ['C

In [5]:
pokemon_tcg_data[['Prefix', 'Suffix']] = pokemon_tcg_data['id'].str.split('-', expand=True)

# Function to format suffixes
def format_suffix(row):
    suffix = row['Suffix']
    
    if suffix[0].isalpha() and suffix[1:].isdigit():  # If suffix starts with a letter followed by digits
        num = int(suffix[1:])  # Extract the numeric portion
        return f"{num:03d}"  # Format as 001, 002, etc.
    elif suffix.isdigit():  # If suffix is purely numeric
        return f"{int(suffix):03d}"  # Format as 001, 002, etc.
    else:
        return suffix  # Leave other cases untouched

# Apply the formatting function to the Suffix column
pokemon_tcg_data['Formatted_Suffix'] = pokemon_tcg_data.apply(format_suffix, axis=1)

# Convert the 'Formatted_Suffix' to numeric if it's purely numeric (for max calculation)
pokemon_tcg_data['Number'] = pd.to_numeric(pokemon_tcg_data['Formatted_Suffix'], errors='coerce')

# Fill NaN values with 0 or a suitable placeholder value before performing max calculations
pokemon_tcg_data['Number'] = pokemon_tcg_data['Number'].fillna(0).astype(int)

# Calculate the maximum number for each prefix group
max_numbers = pokemon_tcg_data.groupby('Prefix')['Number'].transform('max')

# Create the formatted "x of y"
pokemon_tcg_data['Formatted'] = pokemon_tcg_data['Number'].apply(lambda x: f"{int(x):03d}") + ' of ' + max_numbers.apply(lambda x: f"{int(x):03d}")

# Now, update New_ID to strictly show "001 of 102" without any prefix
pokemon_tcg_data['New_ID'] = pokemon_tcg_data['Formatted']

In [6]:
# Convert all text in pokemon_cards to title case
pokemon_cards = pokemon_cards.applymap(lambda x: x.title() if isinstance(x, str) else x)

# Convert all text in pokemon_tcg_data to title case
pokemon_tcg_data = pokemon_tcg_data.applymap(lambda x: x.title() if isinstance(x, str) else x)

In [7]:
merged_data = pd.merge(pokemon_tcg_data, pokemon_cards[['Card Number', 'Price £', 'Pokemon', 'Card Type']], 
                       left_on=['name', 'New_ID'], right_on=['Pokemon', 'Card Number'], how='left')

In [8]:
new_order = ['New_ID', 'Card Type', 'name', 'set', 'generation', 'release_date', 'artist', 'types', 'supertype', 'Price £']
df = merged_data[new_order]

In [9]:
df = df.dropna(subset=['Price £'])
df

,New_ID,Card Type,name,set,generation,release_date,artist,types,supertype,Price £
0,001 Of 102,1St Edition,Alakazam,Base,First,1/9/1999,Ken Sugimori,['Psychic'],Pokemon,27.95
1,001 Of 102,Uk Print,Alakazam,Base,First,1/9/1999,Ken Sugimori,['Psychic'],Pokemon,14.95
2,001 Of 102,Shadowless,Alakazam,Base,First,1/9/1999,Ken Sugimori,['Psychic'],Pokemon,7.95
3,001 Of 102,Unlimited,Alakazam,Base,First,1/9/1999,Ken Sugimori,['Psychic'],Pokemon,1.95
4,002 Of 102,1St Edition,Blastoise,Base,First,1/9/1999,Ken Sugimori,['Water'],Pokemon,119.95
...,...,...,...,...,...,...,...,...,...,...
13847,234 Of 234,Standard,Triple Acceleration Energy,Unbroken Bonds,Seventh,5/3/2019,NaN,NaN,Energy,16.99
18099,004 Of 160,Reverse Holo,Tangela,Crown Zenith,Eighth,1/20/2023,Kurumitsu,['Grass'],Pokemon,0.29
18100,004 Of 160,Standard,Tangela,Crown Zenith,Eighth,1/20/2023,Kurumitsu,['Grass'],Pokemon,0.09
18101,005 Of 160,Reverse Holo,Tangrowth,Crown Zenith,Eighth,1/20/2023,Zig,['Grass'],Pokemon,0.39


In [10]:
df.to_csv('merged_pokemon_data.csv', index=False)

In [11]:
set_counts = df['set'].value_counts()

# Find the values that appear less than 10 times
values_to_remove = set_counts[set_counts < 10].index

# Remove rows where 'set' column has these values
df = df[~df['set'].isin(values_to_remove)]

In [12]:
df = df.rename(columns={
    'New_ID': 'Specific_ID',
    'name': 'Card_Name',
    'set': 'Set_Name',
    'generation': 'Generation',
    'release_date': 'Release_Date',
    'artist': 'Artist',
    'types': 'Types',
    'supertype': 'Supertype'
})

In [13]:
df

,Specific_ID,Card Type,Card_Name,Set_Name,Generation,Release_Date,Artist,Types,Supertype,Price £
0,001 Of 102,1St Edition,Alakazam,Base,First,1/9/1999,Ken Sugimori,['Psychic'],Pokemon,27.95
1,001 Of 102,Uk Print,Alakazam,Base,First,1/9/1999,Ken Sugimori,['Psychic'],Pokemon,14.95
2,001 Of 102,Shadowless,Alakazam,Base,First,1/9/1999,Ken Sugimori,['Psychic'],Pokemon,7.95
3,001 Of 102,Unlimited,Alakazam,Base,First,1/9/1999,Ken Sugimori,['Psychic'],Pokemon,1.95
4,002 Of 102,1St Edition,Blastoise,Base,First,1/9/1999,Ken Sugimori,['Water'],Pokemon,119.95
...,...,...,...,...,...,...,...,...,...,...
13843,230 Of 234,Standard,Electromagnetic Radar,Unbroken Bonds,Seventh,5/3/2019,Yoshinobu Saito,NaN,Trainer,9.75
13844,231 Of 234,Standard,Fire Crystal,Unbroken Bonds,Seventh,5/3/2019,Yoshinobu Saito,NaN,Trainer,12.99
13845,232 Of 234,Standard,Metal Core Barrier,Unbroken Bonds,Seventh,5/3/2019,Studio Bora Inc.,NaN,Trainer,4.79
13846,233 Of 234,Standard,Pokegear 3.0,Unbroken Bonds,Seventh,5/3/2019,Studio Bora Inc.,NaN,Trainer,15.99


In [14]:
df.to_csv('CleanedUp_Data.csv', index=False)